In [1]:
import sys
import os
from os.path import dirname, realpath, join
base_dir = dirname(dirname(os.getcwd()))
import itertools
import pandas as pd
from os.path import join
base_dir

sys.path.insert(0, base_dir)
from config_path import PROSTATE_DATA_PATH, PLOTS_PATH, GENE_PATH, PROSTATE_LOG_PATH
from data.data_access import Data

In [2]:
from utils.stats_utils_delong_xu import delong_roc_variance, delong_roc_test
from matplotlib import  pyplot as plt
from utils.stats_utils import score_ci, pvalue, pvalue_stat
from sklearn import metrics
import numpy as np 

In [3]:
def read_predictions():
    all_models_dict = {}
    base_dir = PROSTATE_LOG_PATH
    models_base_dir = join(base_dir , 'compare/onsplit_ML_test')
    models = ['Linear Support Vector Machine ', 'RBF Support Vector Machine ', 'L2 Logistic Regression', 'Random Forest',
              'Adaptive Boosting', 'Decision Tree']

    for i, m in enumerate(models):
        df = pd.read_csv(join(models_base_dir, m + '_data_0_testing.csv'), sep=',', index_col=0, header=0)
        all_models_dict[m] = df

    pnet_base_dir = join(base_dir , 'pnet/onsplit_average_reg_10_tanh_large_testing')
    df_pnet = pd.read_csv(join(pnet_base_dir, 'P-net_ALL_testing.csv'), sep=',', index_col=0, header=0)
    all_models_dict['P-net'] = df_pnet
    return all_models_dict

In [4]:
all_models_dict = read_predictions()

In [5]:
pnet_predictions = all_models_dict['P-net']
labels = pnet_predictions['y'].values.ravel()

In [6]:
all_models_dict['P-net'].head()

,pred,pred_scores,y
01-087MM_BONE,1.0,0.946647,1
01-095N1_LN,0.0,0.127979,1
08-093J1_LN,1.0,0.990789,1
10362,1.0,0.475796,0
AAPC-IP_LG-069-Tumor-SM-3NC72,0.0,0.114404,0


In [7]:
all_models_dict['Linear Support Vector Machine '].head()

,pred,pred_scores,y
01-087MM_BONE,1,0.765949,1
01-095N1_LN,0,0.242726,1
08-093J1_LN,1,0.956951,1
10362,1,0.809066,0
AAPC-IP_LG-069-Tumor-SM-3NC72,0,0.238246,0


In [8]:
score_fun={}
score_fun['Accuracy'] = metrics.accuracy_score
score_fun['Precision'] = metrics.precision_score
score_fun['AUC'] = metrics.roc_auc_score
score_fun['F1'] = metrics.f1_score
score_fun['AUPR'] = metrics.average_precision_score
score_fun['Recall'] = metrics.recall_score


In [9]:
def fdr(p_vals):
#https://stackoverflow.com/questions/25185205/calculating-adjusted-p-values-in-python
    from scipy.stats import rankdata
    ranked_p_values = rankdata(p_vals)
    fdr = p_vals * len(p_vals) / ranked_p_values
    fdr[fdr > 1] = 1

    return fdr

In [10]:
results=[]
pvalue_list=[]
delong_results=[]
for i, (model_name, predictions_df) in enumerate(all_models_dict.items()):
    if model_name!='P-net':
        pred2 = predictions_df['pred_scores'].values.ravel()
        for func_name, func  in score_fun.items():
                
            if func_name in ['AUC', 'AUPR']:
                col_name= 'pred_scores'
            else:
                col_name= 'pred'
                
            pred_pnet= pnet_predictions[col_name].values.ravel()
            pred_model= predictions_df[col_name].values.ravel()
            
            if func_name=='AUC':
                pvalue_ = delong_roc_test(labels, pred_pnet, pred_model)
                pvalue_delong = 10**pvalue_[0][0]/2
                delong_results.append({'measure': 'AUC_DeLong', 'model':model_name, 'pvalue': pvalue_delong}) 
            
            stat_fun= np.median
            p, scores1, scores2, z = pvalue_stat( labels,pred_pnet,pred_model, func,n_bootstraps=2000, two_tailed=False, seed=1234, stat_fun=np.median)
            med_pnet = stat_fun(scores1)
            med_model = stat_fun(scores2)
            stat_fun_diff =  med_pnet - med_model
            results.append({'measure': func_name, 'model':model_name, 'pvalue': p, 'model median': med_model, 'P-NET median':med_pnet, 'Median difference': stat_fun_diff }) 
            
            

In [11]:
results_df = pd.DataFrame(results)

In [12]:
results_df['pvalue_fdr_adjusted']= results_df.groupby('measure')['pvalue'].apply(fdr)

In [13]:
results_df = results_df.set_index(['model', 'measure'])
results_df = results_df.round(3)


In [14]:
results_df

Median difference  P-NET median  \
model                          measure                                      
RBF Support Vector Machine     AUPR                   0.021         0.881   
                               F1                     0.054         0.755   
                               AUC                    0.013         0.928   
                               Recall                 0.151         0.763   
                               Precision             -0.074         0.750   
                               Accuracy               0.010         0.838   
Decision Tree                  AUPR                   0.140         0.881   
                               F1                     0.035         0.755   
                               AUC                    0.072         0.928   
                               Recall                 0.147         0.763   
                               Precision             -0.122         0.750   
                               Accuracy              -0.005         0.838   
L2 Logistic Regression         AUPR                   0.068         0.881   
                               F1                     0.028         0.755   
                               AUC                    0.045         0.928   
                               Recall                 0.091         0.763   
                               Precision             -0.042         0.750   
                               Accuracy               0.005         0.838   
Adaptive Boosting              AUPR                   0.050         0.881   
                               F1                     0.050         0.755   
                               AUC                    0.039         0.928   
                               Recall                 0.195         0.763   
                               Precision             -0.180         0.750   
                               Accuracy              -0.005         0.838   
Linear Support Vector Machine  AUPR                   0.024         0.881   
                               F1                     0.060         0.755   
                               AUC                    0.021         0.928   
                               Recall                 0.151         0.763   
                               Precision             -0.057         0.750   
                               Accuracy               0.015         0.838   
Random Forest                  AUPR                   0.058         0.881   
                               F1                     0.064         0.755   
                               AUC                    0.033         0.928   
                               Recall                 0.155         0.763   
                               Precision             -0.054         0.750   
                               Accuracy               0.015         0.838   

                                          model median  pvalue  \
model                          measure                           
RBF Support Vector Machine     AUPR              0.860   0.205   
                               F1                0.702   0.088   
                               AUC               0.915   0.212   
                               Recall            0.612   0.002   
                               Precision         0.824   0.927   
                               Accuracy          0.828   0.372   
Decision Tree                  AUPR              0.741   0.001   
                               F1                0.720   0.207   
                               AUC               0.856   0.000   
                               Recall            0.615   0.001   
                               Precision         0.872   0.978   
                               Accuracy          0.843   0.602   
L2 Logistic Regression         AUPR              0.813   0.001   
                               F1                0.727   0.232   
                               AUC               0.883   0.

In [15]:
results_df.to_csv('model_comparison_pvalue.csv')

In [16]:
results_delong_df = pd.DataFrame(delong_results)
results_delong_df['pvalue_fdr_adjusted']= results_delong_df.groupby('measure')['pvalue'].apply(fdr)
results_delong_df = results_delong_df.set_index(['model', 'measure'])
results_delong_df=results_delong_df.round(3)
results_delong_df.to_csv('model_comparison_delong_pvalue.csv')
results_delong_df

,,pvalue,pvalue_fdr_adjusted
model,measure,,
RBF Support Vector Machine,AUC_DeLong,0.210,0.210
Decision Tree,AUC_DeLong,0.001,0.006
L2 Logistic Regression,AUC_DeLong,0.007,0.021
Adaptive Boosting,AUC_DeLong,0.023,0.046
Linear Support Vector Machine,AUC_DeLong,0.117,0.140
Random Forest,AUC_DeLong,0.051,0.076


In [17]:

df = pd.DataFrame(results)
df_cross = pd.crosstab( df['model'], df['measure'], values=df.pvalue, aggfunc='first') 

df_cross

measure,AUC,AUPR,Accuracy,F1,Precision,Recall
model,,,,,,
Adaptive Boosting,0.0190,0.0535,0.6090,0.1255,0.9975,0.0000
Decision Tree,0.0000,0.0010,0.6015,0.2070,0.9780,0.0010
L2 Logistic Regression,0.0060,0.0010,0.4525,0.2325,0.8255,0.0660
Linear Support Vector Machine,0.1255,0.1870,0.2985,0.0665,0.8600,0.0020
RBF Support Vector Machine,0.2115,0.2050,0.3725,0.0885,0.9270,0.0015
Random Forest,0.0485,0.0215,0.3095,0.0750,0.8495,0.0035


In [18]:
df_cross_fdr_adjusted = df_cross.copy()
for c in df_cross_fdr_adjusted.columns:
    df_cross_fdr_adjusted[c] = fdr(df_cross_fdr_adjusted[c])

In [19]:
# df_cross_fdr_adjusted.applymap('{:.3f}'.format)
df_cross_fdr_adjusted = df_cross_fdr_adjusted.round(3)
df_cross_fdr_adjusted

measure,AUC,AUPR,Accuracy,F1,Precision,Recall
model,,,,,,
Adaptive Boosting,0.038,0.080,0.609,0.188,0.998,0.000
Decision Tree,0.000,0.004,0.722,0.248,1.000,0.003
L2 Logistic Regression,0.018,0.004,0.679,0.232,1.000,0.066
Linear Support Vector Machine,0.151,0.224,1.000,0.399,1.000,0.003
RBF Support Vector Machine,0.212,0.205,0.745,0.177,1.000,0.003
Random Forest,0.073,0.043,0.928,0.225,1.000,0.004


In [20]:
df_cross_fdr_adjusted.to_csv('df_cross_fdr_adjusted.csv')